In [1]:
! nvidia-smi

Wed May 18 16:13:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:05:00.0 N/A |                  N/A |
| 40%   51C    P0    N/A /  N/A |    491MiB /  2000MiB |     N/A      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:06:00.0 Off |                  N/A |
| 54%   

### Train MLM

In [2]:
model_path = '../input/deberta-v3-small'
model_name = model_path.split('/')[-1]

In [3]:
from transformers import DebertaV2Tokenizer, DebertaV2ForMaskedLM
import torch

In [4]:
tokenizer = DebertaV2Tokenizer.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
model = DebertaV2ForMaskedLM.from_pretrained(model_path)

Some weights of the model checkpoint at ../input/deberta-v3-small were not used when initializing DebertaV2ForMaskedLM: ['mask_predictions.dense.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.bias', 'deberta.embeddings.position_embeddings.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaV2ForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model

In [6]:
print(model)

DebertaV2ForMaskedLM(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0): DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
              (

In [7]:
inputs = tokenizer("I love [MASK].", return_tensors="pt")

for k in inputs.keys():
  print(k, ' - ', inputs[k])

input_ids  -  tensor([[     1,    273,    472, 128000,    323,      2]])
token_type_ids  -  tensor([[0, 0, 0, 0, 0, 0]])
attention_mask  -  tensor([[1, 1, 1, 1, 1, 1]])


In [8]:
with torch.no_grad():
    logits = model(**inputs).logits

mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

In [9]:
predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)

'runway'

In [10]:
import transformers
from datasets import load_dataset
print(transformers.__version__)

4.18.0


In [11]:
datasets = load_dataset("text", data_files={
    "train": '../input/patent_data/train_v2.txt', 
    "validation": '../input/patent_data/valid.txt'
})

Using custom data configuration default-3a45591c4a018f2a
Reusing dataset text (/home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


  0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
datasets["train"][10]

{'text': '&lsqb;0011&rsqb; Fluorescence polarization allows for development of homogenous screening assays, whereby it is not necessary to separate the products from the reaction between the biological compound and the fluorescent substance and the unreacted fluorescent substance. The method of the present invention is a high throughput assay that uses fluorescence polarization to identify inhibitors of nucleic acid ligases and helicases in a panel or library of test compounds, and is based on the comparison of the fluorescence polarization signals.'}

In [13]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [14]:
show_random_elements(datasets["train"])

,text
0,"[0052] Support bars 20 (see FIGS. 8A-8C) are positioned along the length of the frame 106 to prop-up or suspend the covers 12 over the tires (or tracks) 104 of the vehicle 100. The support bars 20 have a mid-portion 54 which spans the width of the frame 106 and have end portions 56 which extend over the tires (or tracks) 104 on either side of the frame 106. The support bars 20 preferably have hexagonal or octagonal cross-sections, although square or round bars may also be used. The support bars 20 are clamped to the frame 106 of the vehicle 100 using any conventional means. The end portions 56 of the support bars 20 have a length sufficient to accommodate the width of the covers 12 to ensure proper coverage of the tires 104. The support bars 20 are protected at their extreme ends with caps 58 made of a suitable material, such as rubber or plastic to avoid possible hazards associated with exposed, sharp edges. In the preferred embodiment, the support bar caps 58 are formed having an opening 60 through the end thereof that is substantially perpendicular to the length of the support bars 20 so that a cable (or the like) 62 can be threaded through the ends of the support bar caps 58 when the cover system 10 is not in use. This creates a type of barrier to prevent objects from becoming trapped between the exposed support bars 20 when the covers 12 are not in use (see FIG. 11A). Alternatively, rather than having a cable 62 threaded through the ends of the support bar caps 58, bungee cords 63 may be used which can hook to the caps 58 themselves (see FIG. 11B)."
1,"For the NBS AD V_info related to the proposed effective power indication transformation, it normally has number of neighbour, BS1D (base stations' IDs), base station EIRP (Equivalent Isotropically Radiated Power) indicators, base station EIRP values, DCD (Downlink Channel Descriptor) configuration change count, UCD (Uplink Channel Descriptor) configuration change count, etc. Among these parameters, base station E[RP indicators and base station EIRP values are the two which could be possible to apply the weights of the effective power to transfer to the effective power indications. DCD and UCD could be used to fl.irther indicate the configuration change. A simplified NBS_AJ) V_info Syntax of this part is shown in Table 5. Here, the previous EIRP value is transferred to the effective EIRP value as the weight of the effective power is applied to the EIRP value."
2,"Secondly, investigated the relation between adaptation and softening temperature, result shows, the glass composition of softening temperature below 450 DEG C is good.In addition, distinguish from wet fastness and the relation between conductive glass composition and softening temperature, softening temperature is less than 500 DEG C and contains V 2o 5the wet fastness of conductive glass composition good."
3,"In addition, when the fire extinguisher takes the nozzle injection method, it is preferable that the light-receiving part of the lantern is connected to the lid covering the oxygen tank."
4,"The squeeze die-casting process of the present invention may preferably be carried out on what is known in the trade as a VERTI-CAST machine to be described hereinafter. However, it is believed that the process can be carried out with equal efficiency on horizontal casting machines that have been modified for vacuum die evacuation ladling. In vertical casting machines, modified in accordance with the present invention, low temperature metal is drawn by vacuum (vacuum ladled) from the adjacent furnace through the transfer sleeve into the vertically extending shot sleeve to be driven by the vertically upwardly driven plunger to feed the mold cavities through the metal feed gates and runner system arranged concentrically about the center of the shot sleeve. The low temperature metal is driven under pressure applied by the plunger at high velocity through a small feed gate into the evacuated mold cavities. After the m

In [15]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [16]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=12, remove_columns=["text"])

Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-7c77b8ef2b1ba771.arrow


Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-970fc9eb5f3e452b.arrow


Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-352e5cbfde7f9eeb.arrow


Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-d493c843b3a63199.arrow


Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-3ab587b9d2c97ed6.arrow


...

Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6e82519d90b20b97.arrow


Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-2fcd29804764f815.arrow


Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-85576e3131d878fc.arrow


Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-59229526e4394093.arrow


Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-4ccbffc223ef2ca7.arrow


In [17]:
tokenized_datasets["train"][1]

{'input_ids': [1,
  429,
  1406,
  36220,
  1285,
  346,
  528,
  445,
  974,
  3604,
  4553,
  1285,
  346,
  82229,
  3584,
  10109,
  14505,
  261,
  1601,
  4237,
  100366,
  268,
  263,
  45044,
  53650,
  261,
  612,
  266,
  1923,
  985,
  267,
  40821,
  3584,
  18777,
  261,
  61604,
  2211,
  261,
  263,
  5544,
  49323,
  260,
  26252,
  27926,
  57248,
  262,
  79794,
  265,
  40821,
  3584,
  20393,
  5296,
  3714,
  482,
  847,
  8667,
  12752,
  976,
  405,
  283,
  40821,
  3584,
  18777,
  263,
  49323,
  261,
  263,
  267,
  1438,
  264,
  20393,
  5296,
  1920,
  293,
  847,
  2543,
  2040,
  260,
  4237,
  45044,
  53650,
  20170,
  4237,
  283,
  465,
  265,
  262,
  18777,
  568,
  261,
  263,
  264,
  5739,
  3664,
  263,
  17433,
  265,
  777,
  20393,
  61661,
  260,
  279,
  3688,
  4866,
  265,
  40821,
  3584,
  100366,
  268,
  263,
  45044,
  53650,
  303,
  3325,
  331,
  4345,
  478,
  119030,
  271,
  48951,
  28112,
  263,
  7666,
  65925,
  260,
  345

In [18]:
block_size = 128

In [19]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [20]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=12,
)

Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-2f51f8bfbe6511a3.arrow


Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-4d4ab2f4ca97c8a7.arrow


Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-15b0970813f90f33.arrow


Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-ce6698302a68913b.arrow


Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-5d277a8d118df3f8.arrow


...

Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-3a81cbf3c41cab02.arrow


Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6b4a8a252eb3a16b.arrow


Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-deafeca51475bba0.arrow


Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-cb70ab007fd4adb5.arrow


Loading cached processed dataset at /home/user/.cache/huggingface/datasets/text/default-3a45591c4a018f2a/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-a0380cab0ebfecb7.arrow


In [21]:
from transformers import Trainer, TrainingArguments

In [22]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [23]:
training_args = TrainingArguments(
    f"{model_name}+finetuned-patent",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    weight_decay=0.01,
    per_device_train_batch_size=16,
    push_to_hub=False,
    report_to=None,
    num_train_epochs=1, 
    save_total_limit=2,
    save_steps=100000, 
    ignore_data_skip=False
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [25]:
trainer.train('deberta-v3-small+finetuned-patent/checkpoint-200000')

Loading model from deberta-v3-small+finetuned-patent/checkpoint-100000).
/opt/anaconda3/envs/paraphrase/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4521354
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 282585
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 0
  Continuing training from global step 100000
  Will skip the first 0 epochs then the first 100000 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will res

  0%|          | 0/100000 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss
0,0.001700,0.002940


Saving model checkpoint to deberta-v3-small+finetuned-patent/checkpoint-200000
Configuration saved in deberta-v3-small+finetuned-patent/checkpoint-200000/config.json
Model weights saved in deberta-v3-small+finetuned-patent/checkpoint-200000/pytorch_model.bin


KeyboardInterrupt: 

In [26]:
eval_results = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 30156
  Batch size = 8


In [28]:
eval_results

{'eval_loss': 0.0029402177315205336}

In [27]:
import math
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 1.00
